In [186]:
import requests, json
from pprint import pprint
import pandas as pd
from tqdm.auto import tqdm
import random as rd
import copy as cp

In [187]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
#pprint(r, indent=2, depth=1, compact=True)

# get player data from 'elements' field
players = r['elements']

# show data for first player
#pprint(players[0])
pd.set_option('display.max_columns', None)

# create players dataframe
players = pd.json_normalize(r['elements'])
num_players = len(players)
teams = pd.json_normalize(r['teams'])

positions = pd.json_normalize(r['element_types'])
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop([ 'team','id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)
players=players.rename(columns={'name':'team','singular_name_short':'position'})





In [188]:
name_to_id={} #gives player id of the player

for index, row in players.iterrows():
    first = row['first_name']
    second = row['second_name']
    name =first+ " "+ second
    name_to_id[name]=row["id_player"]
   
#col: id_player, first_name, second_name, web_name, team, position
#player database by position
gk = players[players['position']=='GKP']
defender = players[players['position']=='DEF']
mid = players[players['position']=='MID']
fwd = players[players['position']=='FWD']


In [189]:
temp = requests.get(base_url + 'element-summary/' + '1' + '/').json()
player1 = pd.json_normalize(temp['history'])

#number of rounds this season
num_rounds = len(player1)

#gets player's position
def player_position(player_id):
    position = players.loc[players['id_player'] == player_id,'position'].iloc[0]
    return position



#total points from the round for the specific player
def get_round_points(player_id,gameweek): #player_id, gameweek 
    if gameweek>num_rounds:
        assert "Gameweek is not available"
    '''get all gameweek info for a given player_id'''
    
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
   
    row = gameweek -1
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])
    total_points = df["total_points"]
    try:
        points = int(total_points[gameweek])
    except KeyError:
        points = 0
    return points



#player's current price in that round

def player_price(player_id,gameweek):
    if gameweek>num_rounds:
        assert "Gameweek is not available"
    
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    row = gameweek -1
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])
    costs = df["value"]
    if gameweek==1:
        print(player_id)
    try:
        price = costs[gameweek]
    except KeyError:
        price = 45
    return price



In [190]:
#budget = 1000
starter = []
sub = []
#five defenders, five mid, three forward, two goalkeepers
num_gk = len(gk)-1
gk_selection = []
for i in range(2):
    number = rd.randint(0, num_gk)
    gk_price = player_price(gk.iloc[number,0],0)
    while gk_price>55 or number in gk_selection:
        number = rd.randint(0, num_gk)
        gk_price = player_price(gk.iloc[number,0],0)
    gk_selection.append(number)
    if i==0:
        starter.append(gk.iloc[number,0])
    else:
        sub.append(gk.iloc[number,0])

num_df = len(defender)-1
df_selection = []
for i in range(5):
    number = rd.randint(0, num_df)
    df_price = player_price(defender.iloc[number,0],0)
    while df_price>60 or number in df_selection:
        number = rd.randint(0, num_df)
        df_price = player_price(defender.iloc[number,0],0)
    if i<=3:
        starter.append(defender.iloc[number,0])
    else:
        sub.append(defender.iloc[number,0])

num_mid = len(mid)-1
mid_selection = []
for i in range(5):
    number = rd.randint(0, num_mid)
    mid_price = player_price(mid.iloc[number,0],0)
    while mid_price>60 or number in mid_selection:
        number = rd.randint(0, num_mid)
        mid_price = player_price(mid.iloc[number,0],0)
    if i<=3:
        starter.append(mid.iloc[number,0])
    else:
        sub.append(mid.iloc[number,0])


num_fwd = len(fwd)-1
fwd_selection = []
for i in range(3):
    number = rd.randint(0, num_fwd)
    fwd_price = player_price(fwd.iloc[number,0],0)
    while df_price>60 or number in fwd_selection:
        number = rd.randint(0, num_fwd)
        fwd_price = player_price(fwd.iloc[number,0],0)
    if i<2:
        starter.append(fwd.iloc[number,0])
    else:
        sub.append(fwd.iloc[number,0])

print(starter)
print(sub)

[559, 133, 562, 16, 437, 335, 459, 298, 328, 235, 78]
[596, 198, 306, 180]


In [191]:

player_position(starter[0])
for i in range(len(starter)):  
    print(player_position(starter[i]))
for i in range(len(sub)):
    print(player_position(sub[i]))

GKP
DEF
DEF
DEF
DEF
MID
MID
MID
MID
FWD
FWD
GKP
DEF
MID
FWD


In [192]:
avail_round = num_rounds - 1 
cur_team = starter
cur_sub = sub
print(cur_team)


[559, 133, 562, 16, 437, 335, 459, 298, 328, 235, 78]


In [194]:
def mdp(avail_round,team,sub, cur_week):
    
    if cur_week >= avail_round:
        return 0,[]
    teamcost = 0
    for player in team:
        teamcost += player_price(player,cur_week)
    for subs in sub:
        teamcost += player_price(subs,cur_week)
    budget = 1000 - teamcost
    max_value = []
    temp = cp.deepcopy(team)
    print(temp)
    next_week = cur_week+1
    for player in temp:
        max_point = 0 
        max_player = None
        future_changes =[]
        if player_position(player) == "GKP":
            for index, row in gk.iterrows():
                temp1 = cp.deepcopy(temp)
                players = row["id_player"]
                if players in temp:
                    continue
                elif players in sub:
                    
                    reward = get_round_points(players,next_week)
                    
                    temp_sub = cp.deepcopy(sub)
                    temp_sub.remove(players)
                    temp_sub.append(player)
                    temp1.remove(player)
                    temp1.append(players)
                   
                    future_val,best_player = mdp(avail_round,temp1,temp_sub,next_week)
                    future_val*=0.5
                    if reward + future_val > max_point:
                        max_point = reward + future_val
                        max_player = players
                        future_changes = best_player
                        
                elif player_price(players,cur_week) > player_price(player,cur_week) +budget:
                    continue

                else:
                    reward = get_round_points(players,next_week)
                    
                    temp1.remove(player)
                    temp1.append(players)
                    future_val,best_player = mdp(avail_round,temp1,sub,next_week)
                    future_val*=0.5
                    if reward + future_val > max_point:
                        max_point = reward + future_val
                        max_player = players
                        future_changes = best_player
                        
        elif player_position(player) == "DEF":
            
            for index, row in defender.iterrows():
                temp1 = cp.deepcopy(temp)
                players = row["id_player"]
                if players in temp:
                    continue
                elif players in sub:
                    reward = get_round_points(players,next_week)
                    
                    temp_sub = cp.deepcopy(sub)
                    temp_sub.remove(players)
                    temp_sub.append(player)
                    temp1.remove(player)
                    temp1.append(players)
                    future_val,best_player= mdp(avail_round,temp1,sub,next_week)
                    future_val*=0.5
                    if reward + future_val > max_point:
                        max_point = reward + future_val
                        max_player = players
                        future_changes = best_player
                        
                elif player_price(players,cur_week) > player_price(player,cur_week) +budget:
                    continue
                else:
                    reward = get_round_points(players,next_week)
                    
                    temp1.remove(player)
                    temp1.append(players)
                    future_val,best_player = mdp(avail_round,temp1,sub,next_week)
                    future_val*=0.5
                    if reward + future_val > max_point:
                        max_point = reward + future_val
                        max_player = players
                        future_changes = best_player
                        
        elif player_position(player) == "MID":
            for index, row in mid.iterrows():
                temp1 = cp.deepcopy(temp)
                players = row["id_player"]
                if players in temp:
                    continue
                elif players in sub:
                    reward = get_round_points(players,next_week)
                    
                    temp_sub = cp.deepcopy(sub)
                    temp_sub.remove(players)
                    temp_sub.append(player)
                    temp1.remove(player)
                    temp1.append(players)
                    future_val,best_player = mdp(avail_round,temp1,sub,next_week)
                    future_val*=0.5
                    if reward + future_val > max_point:
                        max_point = reward + future_val
                        max_player = players
                        future_changes = best_player
                        
                elif player_price(players,cur_week) > player_price(player,cur_week) +budget:
                    continue
                else:
                    reward = get_round_points(players,next_week)
                    
                    temp1.remove(player)
                    temp1.append(players)
                    future_val,best_player = mdp(avail_round,temp1,sub,next_week)
                    future_val*=0.5
                    if reward + future_val > max_point:
                        max_point = reward + future_val
                        max_player = players
                        future_changes = best_player
                        
        else:
            for index, row in fwd.iterrows():
                temp1 = cp.deepcopy(temp)
                players = row["id_player"]
                if players in temp:
                    continue
                elif players in sub:
                    reward = get_round_points(players,next_week)
                    
                    temp_sub = cp.deepcopy(sub)
                    temp_sub.remove(players)
                    temp_sub.append(player)
                    temp1.remove(player)
                    temp1.append(players)
                    future_val,best_player = mdp(avail_round,temp1,sub,next_week)
                    future_val*=0.5
                    if reward + future_val > max_point:
                        max_point = reward + future_val
                        max_player = players
                        future_changes = best_player
                        
                elif player_price(players,cur_week) > player_price(player,cur_week) +budget:
                    continue
                else:
                    reward = get_round_points(players,next_week)
                    
                    temp1.remove(player)
                    temp1.append(players)
                    future_val,best_player = mdp(avail_round,temp1,sub,next_week)
                    future_val*=0.5
                    if reward + future_val > max_point:
                        max_point = reward + future_val
                        max_player = players
                        future_changes = best_player
        cur_change = (player,max_player)
        future_changes.append(cur_change)
                        
        max_value.append((max_point,future_changes))
    best_team = max(max_value,key=lambda item:item[0])
    return best_team[0], best_team[1]
 
print(mdp(1,cur_team,cur_sub,0))

[559, 133, 562, 16, 437, 335, 459, 298, 328, 235, 78]
